In [93]:
#Accidentalidad Madrid part 2

In [94]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

In [95]:
df=pd.read_csv('df_part1.csv')
test=pd.read_csv('test_part1.csv')

## Arranging the data

In [96]:
# now we are going to extract the percentage of accidents per district
aux=df.groupby(['distrito'])['dow'].count().reset_index().rename(columns={'dow': "num_accidentes"})
aux['peso_distrito']=aux['num_accidentes']*100/aux['num_accidentes'].sum()

# now we merge these values into df and test
df1=pd.merge(df,aux[['distrito','peso_distrito']], on='distrito')
test1=pd.merge(test, aux[['distrito','peso_distrito']], on='distrito')

In [97]:
df1.isnull().sum()

dow                         0
time_int                    0
distrito                    0
tipo_accidente              0
tipo_persona                0
rango_edad                  0
sexo                        0
cod_lesividad           14802
lesividad               14802
coordenada_x_utm            2
coordenada_y_utm            2
positiva_alcohol          102
num_implicados              0
despejado_checked           0
lluviad_checked             0
lluviai_checked             0
nieve_checked               0
granizo_checked             0
nublado_nchecked            0
desconocido_nchecked        0
cruce                       0
vehi__ciclomotor            0
vehi__furgoneta             0
vehi__resto                 0
vehi__turismo               0
peso_distrito               0
dtype: int64

In [98]:
#We are going to get the percentage of accidents by age and by time slot (low, high, night) 

In [99]:
lista_edad=df1['rango_edad'].unique()
grupoe=[]
denoche=[]
depunta=[]
devalle=[]
for i in lista_edad:
    grupoe.append(i)
    edad=df1[df1['rango_edad']==i]
    noche=edad[edad['time_int']=='night']
    denoche.append(noche['dow'].count())
    punta=edad[edad['time_int']=='high']
    depunta.append(punta['dow'].count())
    valle=edad[edad['time_int']=='low']
    devalle.append(valle['dow'].count())
    
rang_ed_horas=pd.DataFrame(list(zip(grupoe,denoche,depunta,devalle)),columns=['rango_edad','night','high','low'])
rang_ed_horas=rang_ed_horas.sort_values('rango_edad',ascending=True)
rang_ed_horas=rang_ed_horas.reset_index(drop=True)
rang_ed_horas

,rango_edad,night,high,low
0,De 10 a 14 años,12,58,235
1,De 15 a 17 años,30,45,176
2,De 18 a 20 años,176,137,665
3,De 21 a 24 años,375,290,1565
4,De 25 a 29 años,416,535,2493
5,De 30 a 34 años,318,562,2484
6,De 35 a 39 años,308,627,2401
7,De 40 a 44 años,271,618,2517
8,De 45 a 49 años,218,633,2243
9,De 50 a 54 años,184,492,1867


In [100]:
# now we get the percentage
rang_ed_horas['peso_night']=rang_ed_horas['night']*100/(rang_ed_horas['night']+rang_ed_horas['high']+rang_ed_horas['low'])
rang_ed_horas['peso_high']=rang_ed_horas['high']*100/(rang_ed_horas['night']+rang_ed_horas['high']+rang_ed_horas['low'])
rang_ed_horas['peso_low']=rang_ed_horas['low']*100/(rang_ed_horas['night']+rang_ed_horas['high']+rang_ed_horas['low'])
rang_ed_horas

,rango_edad,night,high,low,peso_night,peso_high,peso_low
0,De 10 a 14 años,12,58,235,3.934426,19.016393,77.049180
1,De 15 a 17 años,30,45,176,11.952191,17.928287,70.119522
2,De 18 a 20 años,176,137,665,17.995910,14.008180,67.995910
3,De 21 a 24 años,375,290,1565,16.816143,13.004484,70.179372
4,De 25 a 29 años,416,535,2493,12.078978,15.534262,72.386760
5,De 30 a 34 años,318,562,2484,9.453032,16.706302,73.840666
6,De 35 a 39 años,308,627,2401,9.232614,18.794964,71.972422
7,De 40 a 44 años,271,618,2517,7.956547,18.144451,73.899002
8,De 45 a 49 años,218,633,2243,7.045895,20.458953,72.495152
9,De 50 a 54 años,184,492,1867,7.235549,19.347228,73.417224


In [101]:
# we apply the results with the procentages of accidents per age and per time slot to the df
df1['peso_time_int&age']=df1.apply(lambda dfr:  rang_ed_horas[rang_ed_horas['rango_edad']==dfr['rango_edad']]
['peso_'+dfr['time_int']].iloc[0],axis=1 )

#we do the same for test
test1['peso_time_int&age']=test1.apply(lambda dfr:  rang_ed_horas[rang_ed_horas['rango_edad']==dfr['rango_edad']]
['peso_'+dfr['time_int']].iloc[0],axis=1 )

In [102]:
#we check that the number of columns between train (df) and test remain the same
print(df1.shape)
print(test1.shape)

(32431, 27)
(41780, 27)


In [103]:
# We are going to get the percentage of accidents by type of accident and by time slot (low, high, night) 

In [104]:
tipo_acc=[]
night_acc=[]
high_acc=[]
low_acc=[]
for i in df1['tipo_accidente'].unique():
    tipo_acc.append(i)
    acc=df1[df1['tipo_accidente']==i]
    night=acc[acc['time_int']=='night']
    night_acc.append(night['dow'].count())
    high=acc[acc['time_int']=='high']
    high_acc.append(high['dow'].count())
    low=acc[acc['time_int']=='low']
    low_acc.append(low['dow'].count())
    
TA_TS=pd.DataFrame(list(zip(tipo_acc,night_acc,high_acc,low_acc)),columns=['tipo_accidente','night_acc','high_acc','low_acc'])
TA_TS

,tipo_accidente,night_acc,high_acc,low_acc
0,Choque contra obstáculo fijo,1279,517,2871
1,Alcance,475,1332,5487
2,Colisión fronto-lateral,680,1301,6104
3,Colisión lateral,285,832,3267
4,Colisión múltiple,268,515,1448
5,Caída,181,341,1596
6,Atropello a persona,128,462,1539
7,Colisión frontal,76,163,660
8,Solo salida de la vía,39,17,95
9,Atropello a animal,5,15,55


In [105]:
# now we get the percentages
TA_TS['peso_night']=TA_TS['night_acc']*100/(TA_TS['night_acc']+TA_TS['high_acc']+TA_TS['low_acc'])
TA_TS['peso_high']=TA_TS['high_acc']*100/(TA_TS['night_acc']+TA_TS['high_acc']+TA_TS['low_acc'])
TA_TS['peso_low']=TA_TS['low_acc']*100/(TA_TS['night_acc']+TA_TS['high_acc']+TA_TS['low_acc'])
TA_TS

,tipo_accidente,night_acc,high_acc,low_acc,peso_night,peso_high,peso_low
0,Choque contra obstáculo fijo,1279,517,2871,27.405185,11.077780,61.517034
1,Alcance,475,1332,5487,6.512202,18.261585,75.226213
2,Colisión fronto-lateral,680,1301,6104,8.410637,16.091528,75.497835
3,Colisión lateral,285,832,3267,6.500912,18.978102,74.520985
4,Colisión múltiple,268,515,1448,12.012550,23.083819,64.903631
5,Caída,181,341,1596,8.545798,16.100094,75.354108
6,Atropello a persona,128,462,1539,6.012212,21.700329,72.287459
7,Colisión frontal,76,163,660,8.453838,18.131257,73.414905
8,Solo salida de la vía,39,17,95,25.827815,11.258278,62.913907
9,Atropello a animal,5,15,55,6.666667,20.000000,73.333333


In [106]:
df1

,dow,time_int,distrito,tipo_accidente,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,num_implicados,despejado_checked,lluviad_checked,lluviai_checked,nieve_checked,granizo_checked,nublado_nchecked,desconocido_nchecked,cruce,vehi__ciclomotor,vehi__furgoneta,vehi__resto,vehi__turismo,peso_distrito,peso_time_int&age
0,3,low,CIUDAD LINEAL,Choque contra obstáculo fijo,Conductor,De 21 a 24 años,Hombre,NaN,NaN,"444578,153","4475148,102",N,2,1,0,0,0,0,0,0,0,0,0,0,1,6.558540,70.179372
1,3,low,CIUDAD LINEAL,Choque contra obstáculo fijo,Conductor,De 25 a 29 años,Mujer,NaN,NaN,"444578,153","4475148,102",N,2,1,0,0,0,0,0,0,0,0,0,1,0,6.558540,72.386760
2,3,low,CIUDAD LINEAL,Choque contra obstáculo fijo,Conductor,Desconocido,Desconocido,NaN,NaN,"444371,654","4475467,674",N,2,1,0,0,0,0,0,0,0,0,0,0,1,6.558540,62.887646
3,3,low,CIUDAD LINEAL,Choque contra obstáculo fijo,Conductor,Desconocido,Desconocido,NaN,NaN,"444371,654","4475467,674",N,2,1,0,0,0,0,0,0,0,0,0,0,1,6.558540,62.887646
4,3,low,CIUDAD LINEAL,Alcance,Conductor,De 21 a 24 años,Hombre,NaN,NaN,"444057,114","4478208,796",N,2,1,0,0,0,0,0,0,0,0,0,0,1,6.558540,70.179372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32426,2,low,BARAJAS,Atropello a persona,Peatón,Más de 74 años,Mujer,2.0,Ingreso inferior o igual a 24 horas,"450997,492","4480387,879",N,2,1,0,0,0,0,0,0,0,0,0,0,1,1.680491,84.042553
32427,2,low,BARAJAS,Choque contra obstáculo fijo,Conductor,De 55 a 59 años,Hombre,NaN,NaN,"451102,438","4480483,626",N,2,1,0,0,0,0,0,0,0,0,0,0,1,1.680491,73.894231
32428,2,low,BARAJAS,Choque contra obstáculo fijo,Conductor,Desconocido,Desconocido,NaN,NaN,"451102,438","4480483,626",N,2,1,0,0,0,0,0,0,0,0,0,0,1,1.680491,62.887646
32429,2,high,BARAJAS,Colisión lateral,Conductor,De 50 a 54 años,Hombre,NaN,NaN,"450858,441","4480213,111",S,2,1,0,0,0,0,0,0,0,0,0,0,1,1.680491,19.347228


In [107]:
test1.isnull().sum()

dow                         0
time_int                    0
distrito                    0
tipo_accidente              4
tipo_persona                3
rango_edad                  0
sexo                        0
cod_lesividad           18962
lesividad               18962
coordenada_x_utm            0
coordenada_y_utm            0
positiva_alcohol          167
num_implicados              0
despejado_checked           0
lluviad_checked             0
lluviai_checked             0
nieve_checked               0
granizo_checked             0
nublado_nchecked            0
desconocido_nchecked        0
cruce                       0
vehi__ciclomotor            0
vehi__furgoneta             0
vehi__resto                 0
vehi__turismo               0
peso_distrito               0
peso_time_int&age           0
dtype: int64

In [108]:
# we apply the results with the procentages of accidents per accident type and per time slot to the df
df1['peso_time_int&acctype']=df1.apply(lambda dfr:  TA_TS[TA_TS['tipo_accidente']==dfr['tipo_accidente']]
['peso_'+dfr['time_int']].iloc[0],axis=1 )

#we do the same for test
test1['peso_time_int&acctype']=test1.apply(lambda dfr:  TA_TS[TA_TS['tipo_accidente']==dfr['tipo_accidente']]
['peso_'+dfr['time_int']].iloc[0] if pd.notnull(dfr['tipo_accidente']) else np.nan ,axis=1 )

In [109]:
print(df1.shape)
print(test1.shape)

(32431, 28)
(41780, 28)


### One hot encoding and removing not useful columns

In [110]:
df2=pd.get_dummies(df1,columns=['dow','distrito','time_int','tipo_accidente','tipo_persona','rango_edad','sexo','positiva_alcohol'])
test2=pd.get_dummies(test1,columns=['dow','distrito','time_int','tipo_accidente','tipo_persona','rango_edad','sexo','positiva_alcohol'])

In [111]:
print(df2.shape)
print(test2.shape)

(32431, 90)
(41780, 90)


In [112]:
df2.isnull().sum()

cod_lesividad                                  14802
lesividad                                      14802
coordenada_x_utm                                   2
coordenada_y_utm                                   2
num_implicados                                     0
despejado_checked                                  0
lluviad_checked                                    0
lluviai_checked                                    0
nieve_checked                                      0
granizo_checked                                    0
nublado_nchecked                                   0
desconocido_nchecked                               0
cruce                                              0
vehi__ciclomotor                                   0
vehi__furgoneta                                    0
vehi__resto                                        0
vehi__turismo                                      0
peso_distrito                                      0
peso_time_int&age                             

In [113]:
df3=df2.drop(['coordenada_x_utm','coordenada_y_utm','cod_lesividad'], axis=1)
test3=test2.drop(['coordenada_x_utm','coordenada_y_utm','cod_lesividad'], axis=1)

In [121]:
df3.isnull().sum()

lesividad                                      14802
num_implicados                                     0
despejado_checked                                  0
lluviad_checked                                    0
lluviai_checked                                    0
nieve_checked                                      0
granizo_checked                                    0
nublado_nchecked                                   0
desconocido_nchecked                               0
cruce                                              0
vehi__ciclomotor                                   0
vehi__furgoneta                                    0
vehi__resto                                        0
vehi__turismo                                      0
peso_distrito                                      0
peso_time_int&age                                  0
peso_time_int&acctype                              0
dow_0                                              0
dow_1                                         

In [115]:
# The objective column 'lesividad' has 14802 null values (NaN). We will set all these NaN values into the "Se desconoce" category.
df4=df3.copy()
test4=test3.copy()
df4['lesividad']=df4['lesividad'].apply(lambda x:'Se desconoce' if not pd.notnull(x) else x)
test4['lesividad']=test4['lesividad'].apply(lambda x:'Se desconoce' if not pd.notnull(x) else x)


In [116]:
df4.groupby(['lesividad']).count().iloc[:,0:1]

,num_implicados
lesividad,
Asistencia sanitaria ambulatoria con posterioridad,424
Asistencia sanitaria inmediata en centro de salud o mutua,826
Asistencia sanitaria sólo en el lugar del accidente,4345
Atención en urgencias sin posterior ingreso,904
Fallecido 24 horas,34
Ingreso inferior o igual a 24 horas,1310
Ingreso superior a 24 horas,408
Se desconoce,14805
Sin asistencia sanitaria,9375


In [123]:
df4.to_csv('df_part2.csv',index=False)
test4.to_csv('test_part2.csv',index=False)